## 그레디언트 부스팅 계열의 앙상블 모델
- 대표 모델 : XGBoost, LightGBM
- 추가 설치 및 사용법 숙지 필요

- 설치 여부 확인

In [62]:
import pandas as pd
import xgboost, lightgbm

print(f'xgboost : {xgboost.__version__}')
print(f'lightgbm : {lightgbm.__version__}')

xgboost : 2.0.3
lightgbm : 4.1.0


### [1] 모듈 로딩 및 데이터 준비 <hr>

In [63]:
## 데이터 파일 변수 선언
feature_name_file = '../data/human_activity/features.txt'
label_file = '../data/human_activity/activity_labels.txt'

X_train_file = '../data/human_activity/train/X_train.txt'
y_train_file = '../data/human_activity/train/y_train.txt'

x_test_file = '../data/human_activity/test/X_test.txt'
y_test_file = '../data/human_activity/test/y_test.txt'

In [64]:
## 데이터 로딩
import pandas as pd
import numpy as np

feature_nameDF = pd.read_csv(feature_name_file, header=None, names=['col_id', 'col_name'], sep='\s+')
# labelDF = pd.read_csv(label_file, header=None, names=['col_id', 'col_name'], sep='\s+')
# # labelDF
X_trainDF = pd.read_csv(X_train_file, header=None, sep='\s+')
y_trainDF = pd.read_csv(y_train_file, header=None, sep='\s+')
x_testDF = pd.read_csv(x_test_file, header=None, sep='\s+')
y_testDF = pd.read_csv(y_test_file, header=None, sep='\s+')


In [65]:
# 데이터 확인
feature_nameDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   col_id    561 non-null    int64 
 1   col_name  561 non-null    object
dtypes: int64(1), object(1)
memory usage: 8.9+ KB


In [66]:
## 중복 체크
feature_nameDF.col_name.duplicated().sum()
feature_nameDF

,col_id,col_name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y
...,...,...
556,557,"angle(tBodyGyroMean,gravityMean)"
557,558,"angle(tBodyGyroJerkMean,gravityMean)"
558,559,"angle(X,gravityMean)"
559,560,"angle(Y,gravityMean)"


In [67]:
mask = feature_nameDF[feature_nameDF.col_name.duplicated()].index

feature_nameDF.drop(mask)

,col_id,col_name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y
...,...,...
556,557,"angle(tBodyGyroMean,gravityMean)"
557,558,"angle(tBodyGyroJerkMean,gravityMean)"
558,559,"angle(X,gravityMean)"
559,560,"angle(Y,gravityMean)"


In [68]:
x_testDF

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,0.071645,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.401189,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,0.062891,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,0.116695,-0.344418,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.121711,-0.534685,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,0.310155,-0.053391,-0.099109,-0.287866,-0.140589,-0.215088,-0.356083,-0.148775,-0.232057,0.185361,...,0.074472,-0.376278,-0.750809,-0.337422,0.346295,0.884904,-0.698885,-0.651732,0.274627,0.184784
2943,0.363385,-0.039214,-0.105915,-0.305388,0.028148,-0.196373,-0.373540,-0.030036,-0.270237,0.185361,...,0.101859,-0.320418,-0.700274,-0.736701,-0.372889,-0.657421,0.322549,-0.655181,0.273578,0.182412
2944,0.349966,0.030077,-0.115788,-0.329638,-0.042143,-0.250181,-0.388017,-0.133257,-0.347029,0.007471,...,-0.066249,-0.118854,-0.467179,-0.181560,0.088574,0.696663,0.363139,-0.655357,0.274479,0.181184
2945,0.237594,0.018467,-0.096499,-0.323114,-0.229775,-0.207574,-0.392380,-0.279610,-0.289477,0.007471,...,-0.046467,-0.205445,-0.617737,0.444558,-0.819188,0.929294,-0.008398,-0.659719,0.264782,0.187563


In [69]:
X_trainDF = X_trainDF.T.drop(mask).T
x_testDF = x_testDF.T.drop(mask).T

In [72]:
y_trainDF

,0
0,5
1,5
2,5
3,5
4,5
...,...
7347,2
7348,2
7349,2
7350,2


In [77]:
feature_nameDF.shape, X_trainDF.shape, x_testDF.shape

((561, 2), (7352, 477), (2947, 477))